In [2]:
import requests # to get the HTML
from bs4 import BeautifulSoup # to get parse the text

import numpy as np
import pandas as pd

import tqdm
from datetime import date, timedelta
from selenium.webdriver.common.by import By
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--headless")
# chrome_options.headless = True # also works
driver = webdriver.Chrome(options=chrome_options)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [4]:
pd.options.display.max_colwidth = 200

In [5]:
base_url = 'https://www.jobs.bg/'
url = 'https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&from_hp=1&term=#paging'
#      https://www.jobs.bg/front_job_search.php?frompage=15&add_sh=1&from_hp=1&term=#paging

r = driver.get(url)
html = driver.execute_script("return document.documentElement.outerHTML")
soup = BeautifulSoup(html, 'html.parser')

In [6]:
def get_pages(soup):
    """Get the total number of job offerings and the total number of pages.
    output: total number of pages
    """
    total_job_number = soup.find('td',
                            attrs={'style':
                                  'height:25px;width:220px;font-weight:500;padding-bottom:5px;'}).text.split()[-1]
    total_job_number= int(total_job_number)
    total_pages = int(total_job_number/15)+1 #round up
    return total_pages

In [17]:
# total_pages = get_pages(soup)
total_pages = 30
results =[]

#for page in tqdm.tqdm(range(total_pages)):
for page in tqdm.tqdm(range(49, 50)):

    url = f'https://www.jobs.bg/front_job_search.php?frompage={page*15}&add_sh=1&from_hp=1&term=#paging'
    r = driver.get(url)
    html = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(html, 'html.parser')

    job_infos = soup.find_all("td", attrs = {"class": "offerslistRow"})
    
    for i in range(0, 45, 3):
        text_info_elem = job_infos[i]
        job_logo_elem = job_infos[i+1]
        job_company_elem = job_infos[i+2]
        
        position_id = text_info_elem.find('a',attrs={'class':'card__title mdc-typography mdc-typography--headline6 text-overflow'})['href'].lstrip('job/')
        position_url = base_url+text_info_elem.find('a',attrs={'class':'card__title mdc-typography mdc-typography--headline6 text-overflow'})['href']
        company_name = job_company_elem.text.strip()

        
        position_r = driver.get(position_url)
        html = driver.execute_script("return document.documentElement.outerHTML")
        position_soup = BeautifulSoup(html, 'html.parser')
        
        position_views = position_soup.find('span', string = 'Разглеждания:').parent.text
        
        iframe_elem = position_soup.find('iframe', attrs = {'id':'customJobIframe'})
        raw_html_elem = position_soup.find('td', attrs = {'class':'jobTitle'})
        if iframe_elem:
            iframe_url = iframe_elem.attrs['src']
            iframe_r = driver.get(iframe_url)
            html = driver.execute_script("return document.documentElement.outerHTML")
            iframe_soup = BeautifulSoup(html, 'html.parser')
            job_text = iframe_soup.text.strip()
        elif raw_html_elem: 
            job_text=raw_html_elem.parent.parent.text.strip()
        else:
            job_text = np.nan
        
        job_date = text_info_elem.find('span',attrs={'class':'card__subtitle mdc-typography mdc-typography--overline line-height-1-5'}).text.strip()
        if job_date == 'днес':
            job_date = date.today().strftime("%d.%m.%y")
        elif job_date == 'вчера':
            job_date = (date.today() - timedelta(days=1)).strftime("%d.%m.%y")

        rating_elem = text_info_elem.find('span',attrs={'class':'iconed'})

        if rating_elem:
            rataing_elem_styles = rating_elem["style"].split(";")
            rating_elem_color_styles = [x for x in rataing_elem_styles if "color:" in x]
            rating_color = rating_elem_color_styles[0].split(":")[1]
            rating_full = rating_elem.text.strip().count("\uf4b3")
            rating_half = (rating_elem.text.strip().count("\uf4b1"))/2
            rating_final = rating_full + rating_half
        else:
            rating_color = np.nan
            rating = np.nan
       
        position = text_info_elem.find('a',attrs={'class':'card__title mdc-typography mdc-typography--headline6 text-overflow'}).text.strip()
        benefits = text_info_elem.find('span',attrs={'class':'card__subtitle mdc-typography mdc-typography--body2 top-margin'}).text.strip()
        
        tech_list = []
        for a in text_info_elem.find_all('img'):
            tech = a.attrs['alt']
            tech_list.append(tech)
        tech_list = ", ".join(tech_list)
        results.append([position_id,position_url,company_name,position_views,job_text,job_date, rating_final, rating_color, position, benefits, tech_list])

100%|██████████| 1/1 [00:04<00:00,  4.33s/it]


In [18]:
jobs_df = pd.DataFrame(results, columns=["Position_Id","Position_URL","Company Name","Position Views","Job Description","Date Posted", 
                                         "Position_Rating","Rating Color", "Position", "Benefits", "Technologies"])
jobs_df

,Position_Id,Position_URL,Company Name,Position Views,Job Description,Date Posted,Position_Rating,Rating Color,Position,Benefits,Technologies
0,5670542,https://www.jobs.bg/job/5670542,АБРИТЕС ООД,Разглеждания: 442,"We are a Bulgarian IT company, established in 2005 and specializing in the development and production of its own software and hardware for motor vehicle diagnostics.\n \n ...",28.04.21,5.0,#66c1ff,Graphic Designer,София,"Creative Cloud, Photoshop, Illustrator, InDesign"
1,5670713,https://www.jobs.bg/job/5670713,И:ФАО България ЕООД,Разглеждания: 136,"i:FAO is part of Amadeus IT group, one of the world´s top 10 software companies with a global team of 19,000 professionals of 145 nationalities, working across 190 countries.\nWe are an IT service...",28.04.21,5.0,#66c1ff,iOS Developer,София; Възможност за работа от вкъщи; Дистанционно интервю,"iOS, Jenkins"
2,5670726,https://www.jobs.bg/job/5670726,Modis Bulgaria EOOD,Разглеждания: 731,"Connecting businesses and tech talents in a fast-changing world. This is what describes best the mission of Modis as a global leader in workforce, skilling, and consulting solutions for our partne...",28.04.21,5.0,#66c1ff,Vendor and Order Management Specialist,София; Възможност за работа от вкъщи; Дистанционно интервю,Английски
3,5670656,https://www.jobs.bg/job/5670656,СОФАРМА ТРЕЙДИНГ АД,Разглеждания: 410,"SAP Business Analyst Support Consultant\nWe are Sopharma Trading, part of the Sopharma Group. Our story begins in 2006 with the merge of five distributing companies and today we are a leading heal...",28.04.21,5.0,#66c1ff,SAP Business Analyst Support Consultant,София,SAP
4,5670711,https://www.jobs.bg/job/5670711,Самекс ЕООД / KFC,Разглеждания: 196,Фирма САМЕКС ЕООД оперира ресторантите на KFC в България от 1994 г. Ние представляваме и веригите DUNKIN'DONUTS и NORDSEE и сме първият оператор в сферата на бързото обслужване на българския пазар...,28.04.21,5.0,#66c1ff,Работник в ресторант,Виноградец,
5,5670603,https://www.jobs.bg/job/5670603,МЕНПАУЪР БЪЛГАРИЯ ООД,Разглеждания: 395,"Manpower is part of ManpowerGroup, a Fortune 500 Global Business. We’ve been powered by our people since 1948. That’s 70 years of making dreams reality. We don’t make our people; we are made by th...",28.04.21,5.0,#66c1ff,React Developer,София; Възможност за работа от вкъщи; Заплата от 4500 до 8000 BGN (Нето),"Английски, JavaScript, React"
6,5670688,https://www.jobs.bg/job/5670688,САЙТЕЛ БЪЛГАРИЯ ЕООД,Разглеждания: 159,"Guided by 35+ years of experience and a customer-obsessed approach, Sitel offers tailored customer care solutions and is redefining and transforming the contact center and the associate experience...",28.04.21,5.0,#66c1ff,Turkish and English Speaking Customer Care Specialist,София; Дистанционно интервю,
7,5670672,https://www.jobs.bg/job/5670672,ЕЙЧ АР ЕС СЪРВИСИЗ БЪЛГАРИЯ ООД,Разглеждания: 139,"At HRS, we believe the right job can transform a person's life and the right person can transform a business. We're passionate about connecting our candidates with the right job for them.The exten...",28.04.21,5.0,#66c1ff,Data Engineer,София; Дистанционно интервю,"Английски, Python, SQL, MySQL, PostgreSQL, Tableau, Linux, GCP"
8,5670646,https://www.jobs.bg/job/5670646,Съдърланд Глобъл Сървисиз България ЕООД,Разглеждания: 50,Technical Support Consultant with Norwegian and English for McAfee\n\nInterested in new technology and excited about improving your technical skills? \n Keen about practicing your lan...,28.04.21,5.0,#66c1ff,Technical Support Consultant with Norwegian and English for McAfee,София,
9,5670640,https://www.jobs.bg/job/5670640,Съдърланд Глобъл Сървисиз България ЕООД,Разглеждания: 105,"WE ARE GROWING!\n \nTravel advisor for Hilton fluent in Polish\n\n\n#BookYourCareer\n \nFor those of you who are interested in tourism, we offer you to get closer and work ...",28.04.21,5.0,#66c1ff,Travel advisor for Hilton fluent in Polish and English,София,


In [9]:
job_infos = soup.find_all("td", attrs = {"class": "offerslistRow"})

for i in range(0, 45, 3):
    text_info_elem = job_infos[i]
    job_logo_elem = job_infos[i+1]
    job_company_elem = job_infos[i+2]
   
print(job_company_elem.text.strip())

Съдърланд Глобъл Сървисиз България ЕООД


In [10]:
driver.get(position_url)
html = driver.execute_script("return document.documentElement.outerHTML")
position_soup = BeautifulSoup(html, 'html.parser')

In [11]:
#position_r = requests.get(position_url)
#position_soup = BeautifulSoup(position_r.text, 'html.parser')

position_views = position_soup.find('span', string = 'Разглеждания:').parent

In [10]:
position_views.text

'Разглеждания: 136'

In [18]:
conda list

# packages in environment at /Users/LyubomirKushev/opt/anaconda3:
#
# Name                    Version                   Build  Channel
_anaconda_depends         2020.07                  py38_0  
_ipyw_jlab_nb_ext_conf    0.1.0                    py38_0  
alabaster                 0.7.12             pyhd3eb1b0_0  
anaconda                  custom                   py38_1  
anaconda-client           1.7.2                    py38_0  
anaconda-navigator        2.0.1                    py38_0  
anaconda-project          0.9.1              pyhd3eb1b0_1  
anyio                     2.2.0            py38hecd8cb5_1  
applaunchservices         0.2.1                      py_0  
appnope                   0.1.2           py38hecd8cb5_1001  
appscript                 1.1.2            py38h9ed2024_0  
argh                      0.26.2                   py38_0  
argon2-cffi               20.1.0           py38h9ed2024_1  
asn1crypto                1.4.0                      py_0  
astroid                